In [2]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
import urllib
import os
from tqdm import tqdm 
import re
import time


## Create Data Base to QuetsionPro resposnses

# Connection test

In [ ]:
max_retries = 5           # Max retries: important in cases of instability
base_delay = 1             # Delay: important in cases of system overload
timeout = 60               # Connection timeout in seconds: prevents the program from hanging indefinitely

user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")

server = 'server_name'
database = 'db_name'

for attempt in range(1, max_retries + 1):
    try:
        conn = pyodbc.connect(f"""
        DRIVER={{ODBC Driver 18 for SQL Server}};
        SERVER={server};
        DATABASE={database};
        TrustServerCertificate=Yes;
        MultiSubnetFailover=Yes;
        UID={user};
        PWD={password};
    """)
        print("Connection successful!")
        break
    except pyodbc.Error as e:
        print(f"Attempt {attempt} failed: {e}")
        if attempt == max_retries:
            print("Max retries reached. Aborting.")
            raise
        sleep_time = base_delay * (2 ** (attempt - 1))
        print(f"Waiting {sleep_time} seconds before retrying...")
        time.sleep(sleep_time)


Conexão bem-sucedida!


# Create Table

In [ ]:
for attempt in range(1, max_retries + 1):
    try:
        conn = pyodbc.connect(f"""
        DRIVER={{ODBC Driver 18 for SQL Server}};
        SERVER={server};
        DATABASE={database};
        TrustServerCertificate=Yes;
        MultiSubnetFailover=Yes;
        UID={user};
        PWD={password};
    """)
        print("Connection successful!")
        break
    except pyodbc.Error as e:
        print(f"Attempt {attempt} failed: {e}")
        if attempt == max_retries:
            print("Max retries reached. Aborting.")
            raise
        sleep_time = base_delay * (2 ** (attempt - 1))
        print(f"Waiting {sleep_time} seconds before retrying...")
        time.sleep(sleep_time)
        time.sleep(sleep_time)


excel_file = 'questionpro_data.xlsx'
df = pd.read_excel(excel_file, dtype=str) 

cursor = conn.cursor()

# delete the table if it already exists
cursor.execute("IF OBJECT_ID('TB_QUESTIONPRO_DATA', 'U') IS NOT NULL DROP TABLE TB_QUESTIONPRO_DATA;")
conn.commit()

# Create table with NVARCHAR(MAX) columns
columns_sql = ", ".join([f"[{col}] NVARCHAR(MAX)" for col in df.columns])
create_sql = f"CREATE TABLE TB_QUESTIONPRO_DATA ({columns_sql});"
cursor.execute(create_sql)
conn.commit()

# Insert data
for _, row in df.iterrows():
    placeholders = ", ".join(["?"] * len(row))
    insert_sql = f"INSERT INTO TB_QUESTIONPRO_DATA VALUES ({placeholders})"
    cursor.execute(insert_sql, tuple(row.fillna('').values))
conn.commit()

print("Table created and data inserted successfully")
conn.close()

Conexão bem-sucedida!
Tabela criada e dados inseridos com sucesso.
